In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import time
from random import randint
import string
import pandas as pd
import numpy as np

In [2]:
# Spotify authentication for API calls
client_credentials_manager = SpotifyClientCredentials(client_id='38cd6a8bbed74d03828946532285b9c2', client_secret='0c2aaa4da1a54386b044f315f66ae260')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=4)

## Get Artists using Spotify Api

In [11]:
# initialize list to store artists
artist_results = []
for letter in list(string.ascii_lowercase):
    #set initial offset for search page
    offset = 0
    # loop through search results increasing offset each time
    for i in range(50):
        try:
            #search using letter of alphabet and collect all artist information for that letter
            search = sp.search(q='artist:{}'.format(letter), type='artist', limit=50, offset = offset)
            #add results to list
            artist_results.append(search)
            offset += 50
        except:
            continue

HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.c

HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.c

HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.c

In [12]:
artist_list = []
artist_name = []
# this will loop through each of the results from above and collect artist attributes
for i in range(len(artist_results)):
    for result in artist_results[i]['artists']['items']:
        art_attributes = []
        artist = result['name']
        uri = result['uri'].split(':')[-1]
        genre = result['genres']
        # if we have already added this artist just go to next result
        if artist in artist_name:
            continue
        else:
            artist_name.append(artist)
            art_attributes.append(artist)
            art_attributes.append(uri)
            art_attributes.append(genre)
            artist_list.append(art_attributes)

In [13]:
len(artist_list)

34964

In [15]:
artist_df = pd.DataFrame(artist_list)

In [17]:
artist_df.to_csv('artist_genre.csv')

## Web Scraping

I had many issues trying to get data from the web as many sites do not want you to easily access the lyrics to a song. This is mainly due to copyright issues, I was able to find one site that i was able to acrape some lyrics from. First i had to use the artists name to navigate to their page on the site, this page displays all of the artists songs and other metadata including genre. I scraped this page for all of the links to the artists songs, which i will use to scrape the lyrics, as well as the genres associated with the artist. Last i navigated to every link that i collected and scraped all the lyrics for all the songs i had collected.

In [1]:
#function to find artist links
def get_artist_links(artist):
    links = []
    artist = artist.replace(' ', '_')
    url = 'http://lyrics.wikia.com/wiki/' + artist
    try:    
        content = urllib.request.urlopen(url).read() 
        soup = BeautifulSoup(content, 'html.parser')
        trackography = soup.find('div', class_="mw-content-ltr mw-content-text")
        layer1 = soup.findAll('ol')
        timer1 = time.time()
        for deep in layer1:
            layer2 = deep.findAll('li')
            for deeper in layer2:
                timer2 = time.time()
                if (timer2 - timer1) > 40:
                    break
                else:
                    link = deeper.find('a').get('href')
                    links.append(link)
    except:
        link = 'Exception Occured'
        links.append(link)
    return links

In [3]:
# loops through all of our artists and gathers links for each artist
link_df = []
i=0
for artist in artists['artist']:
    artist = str(artist)
    links = get_artist_links(artist)
    for link in links:
        song_list = []
        song_list.append(artist)
        song_list.append(artists['song_uri'][i])
        song_list.append(artists['genre'][i])
        song_list.append(link)
        link_df.append(song_list)
    i += 1

In [4]:
#function to gather lyrics now that we have links
def get_lyrics_wiki(link):  
    url = 'http://lyrics.wikia.com/' + link
    all_lyrics = []
    try: 
        content = urllib.request.urlopen(url).read() 
        soup = BeautifulSoup(content, 'html.parser') 
        lyrics = str(soup.find('div', class_='lyricbox'))
        lyrics = lyrics.replace('<div class="lyricbox">', ' ')
        lyrics = lyrics.replace('<div class="lyricsbreak">', ' ')
        lyrics = lyrics.replace('</div>', ' ')
        all_lyrics.append(lyrics)
    except Exception as e: 
        all_lyrics.append("Exception occurred")
    return all_lyrics

In [ ]:
#intialize starting conditions
lyrics = []
start = 0
end = 2500

In [ ]:
# this will loop through all links we got and gather song lyrics
# i ran this in 23 different notebooks at the same time in order to
# cut down on time it took to get the lyrics
begin = time.time()
while start != end:
    for link in track_links['link'][start:end]:
        lyric = get_lyrics_wiki(link)
        lyrics.append(lyric)
    start = end
    end +=2500
    if end >= 711002:
        end = 33333
    elapsed = time.time() - begin
    print(end)
    print(elapsed)
    #export each iteration to csv in case kernel dies or we get blocked from lyric wiki site
    lyric_df = pd.DataFrame(lyrics)
    lyric_df.to_csv('lyric_df1.csv') 